In [81]:
"""
{
  [assessment_name]: {
    "description": [description text],
    "url": [reproschema_url],
    "data elements": {
       [element_name]: {
          'description': [description text],
          'datatype': <type>,
          'choices': [
             [choice name],
             [another choice name],
           ],
           'termURL': [reproschema_url]
       }
     } 
   }
}
"""

"""
Ex now:
{
    "adhd_session_id": {
        "description": "Unique identifier for the ADHD session."
    }
}
"""
#dict of non-items only that match
#list of keys that don't correspond to any reproschema files
#
# def
# for each phenotype jsons
#   open as dict
#   for each key
#       recursively search reproschema for item files that match
#       if found at least one:
#           if at least one type item
#               then populate data
#           else
#              add to list of non-items only that match 
#       else:
#           add to list of keys that don't correspond to any reproschema files
#
# def populate data
#           

'\nEx now:\n{\n    "adhd_session_id": {\n        "description": "Unique identifier for the ADHD session."\n    }\n}\n'

In [82]:
import os
import json

b2ai_redcap2rs_activities_dir = "/Users/isaacbevers/sensein/reproschema-wrapper/b2ai-redcap2rs/activities"

def search_string_in_json_files(directory, search_string):
    matching_files = []  # List to store paths of matching JSON files

    # Walk through each directory and file in the given directory
    for root, dirs, files in os.walk(directory):
        for phenotype_file_name in files:
            file_path = os.path.join(root, phenotype_file_name)
            try:
                # Attempt to open and load the file as JSON
                with open(file_path, 'r', encoding='utf-8') as file:
                    data = json.load(file)  # Load JSON data
                    # Check if the search string is present in the JSON content
                    if search_string in str(data) or search_string in str(phenotype_file_name):
                        matching_files.append(file_path)
            except Exception:
                # Skip files that cannot be read or loaded as JSON
                continue
    return matching_files  # Return the list of matching file paths
# search_string_in_json_files(b2ai_redcap2rs_activities_dir, "hello")
# search_string = "adhd_session_id"
# directory = "/Users/isaacbevers/sensein/reproschema-wrapper/b2ai-redcap2rs/activities/q_mood_adhd_adult/items"
# search_string_in_json_files(directory, search_string)


In [93]:
import requests

def is_url_resolvable(url):
    """
    Checks if the URL is resolvable.

    Parameters:
        url (str): The URL to check.

    Returns:
        bool: True if the URL is resolvable, False otherwise.
    """
    try:
        response = requests.get(url)
        return response.status_code == 200
    except requests.exceptions.RequestException:
        return False

def get_reproschema_raw_url(path, checksum="65734f24a32b69ed8dca2e92567cbb580cc0d492", branch="main"):
    """
    Generates a raw GitHub URL for a file in the project.

    Parameters:
        path (str): Path to the file in the project.
        checksum (str): The checksum of the file (default is a specific value).
        branch (str): Branch name (default is 'main').

    Returns:
        str: The raw GitHub URL.
    """
    path = path.split('/b2ai-redcap2rs/', 1)[-1]
    url = f"https://raw.githubusercontent.com/sensein/b2ai-redcap2rs/{checksum}/{path}"
    if is_url_resolvable(url):
        return url
    else:
        return False



In [ ]:

# # def set_data_element():
# def get_reproschema_raw_url(checksum="65734f24a32b69ed8dca2e92567cbb580cc0d492"):
    


def populate_data_element(output_phenotype_dict, key, item_file_path, phenotype_file_name):
    # print(item_file_path)
    with open(item_file_path, 'r', encoding='utf-8') as file:
        reproschema_item = json.load(file)
    if key not in output_phenotype_dict:
        output_phenotype_dict[key] = {}
    if "question" in reproschema_item:
        output_phenotype_dict[key]["question"] = reproschema_item["question"]

    output_phenotype_dict[key]["datatype"] = reproschema_item["responseOptions"]["valueType"]

    if "choices" in reproschema_item["responseOptions"]:
        output_phenotype_dict[key]["choices"] = reproschema_item["responseOptions"]["choices"]
    else:
        output_phenotype_dict[key]["choices"] = None
    reproschema_raw_url = get_reproschema_raw_url(item_file_path)
    if reproschema_raw_url:
        output_phenotype_dict[key]["termURL"] = reproschema_raw_url

    #TODO add URL

    # phenotype_file_dict[phenotype_file_name][""] = # question field

        #       'description': [description text],
        #   'datatype': <type>,
        #   'choices': [
        #      [choice name],
        #      [another choice name],
        #    ],
        #    'termURL': [reproschema_url]

In [85]:
def get_all_schema_paths(directory):
    schema_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('schema'):
                schema_paths.append(os.path.join(root, file))
    return schema_paths


def get_activity_schema_path(item_path):
    activity_dir = os.path.join(item_path.split('/activities/', 1)[0], 'activities', item_path.split('/activities/', 1)[-1].split('/')[0])
    schema_paths = []
    for root, _, files in os.walk(activity_dir):
        for file in files:
            if file.endswith('schema'):
                schema_paths.append(os.path.join(root, file))
    if len(schema_paths) == 1:
        return schema_paths[0]
    else:
        # print(schema_paths)
        raise ValueError(f"Wrong number of schema paths: {len(schema_paths)}")

    

In [95]:
import copy 
            

matching_non_item_files = {}
multiple_item_files = []
non_matching = []

# Specify the phenotype_dir containing the .json files
phenotype_dir = "/Users/isaacbevers/sensein/b2ai-wrapper/b2aiprep/src/b2aiprep/prepare/resources/b2ai-data-bids-like-template/phenotype"
# Dictionary to store the loaded JSON data

def main():
    # Loop through each file in the phenotype_dir
    for phenotype_file_name in os.listdir(phenotype_dir):
        # Check if the file ends with .json and is not "<measurement_tool_name>.json"
        if phenotype_file_name.endswith(".json") and phenotype_file_name != "<measurement_tool_name>.json":
            file_path = os.path.join(phenotype_dir, phenotype_file_name)


            with open(file_path, 'r', encoding='utf-8') as file:
                phenotype_file_dict = json.load(file)

            # unnest to make output idempotent
            if len(phenotype_file_dict) == 1: 
                key = next(iter(phenotype_file_dict))
                if "data_elements" in phenotype_file_dict[key]:
                    phenotype_file_dict = phenotype_file_dict[key]["data_elements"]

            activity_schema_path = ""
            output_phenotype_dict = copy.deepcopy(phenotype_file_dict) 
            for key in phenotype_file_dict:
                if '___' in key:
                    key = key.split('___')[0]
                    output_phenotype_dict[key] = {}
                file_paths = search_string_in_json_files(b2ai_redcap2rs_activities_dir, key)
                if file_paths:
                    item_file_paths = [path for path in file_paths if "item" in path]
                    if item_file_paths and len(item_file_paths) == 1:
                        # populate_data_element(output_phenotype_dict, key, item_file_paths[0], phenotype_file_name)
                        if not activity_schema_path:
                            activity_schema_path = get_activity_schema_path(item_file_paths[0])
                    elif item_file_paths and len(item_file_paths) > 1:
                        # select the correct one
                        for path in item_file_paths:
                            if os.path.basename(path) == key:
                                if not activity_schema_path:
                                    activity_schema_path = get_activity_schema_path(path)
                                # populate_data_element(output_phenotype_dict, key, path, phenotype_file_name)
                        multiple_item_files.append(item_file_paths)
                    else:
                        matching_non_item_files[key] = file_paths
                else:
                    non_matching.append(key)
            print(activity_schema_path)

            activity_schema_name = os.path.basename(activity_schema_path)
            output_phenotype_dict = {"data_elements": output_phenotype_dict}
            output_phenotype_dict["url"] = get_reproschema_raw_url(activity_schema_path)
            output_phenotype_dict = {activity_schema_name: output_phenotype_dict}
            # TODO
            # output_phenotype_dict[phenotype_file_name]["url"] = 
            #phenotype_file_dict[phenotype_file_name][data elements] = 
            # if phenotype_file_name not in output_phenotype_dict:
            #     output_phenotype_dict = {phenotype_file_name: RS ASSESSMENT NAME}
            # print(output_phenotype_dict)
            with open(file_path, 'w', encoding='utf-8') as file:
                json.dump(output_phenotype_dict, file, ensure_ascii=False, indent=4)
"""
{
  [assessment_name]: {
    "description": [description text],
    "url": [reproschema_url],
    "data elements": {
"""
            # save data

#       if found at least one:
#           if at least one type item
#               then populate data
#           else
#              add to list of non-items only that match 
#       else:
#           add to list of keys that don't correspond to any reproschema files
        
main()



['vhi_session_id', 'vhi_duration', 'voice_difficult_hear', 'tough_to_understand', 'voice_restrict_social', 'left_out_convo', 'voice_lose_income', 'strain_voice', 'voice_clarity', 'voice_upsetting', 'voice_handicapped', 'ask_whats_wrong_voice']
/Users/isaacbevers/sensein/reproschema-wrapper/b2ai-redcap2rs/activities/q_generic_voice_handicap_index_vhi10/q_generic_voice_handicap_index_vhi10_schema
['diagnosis_vfp_gsd', 'diagnosis_vfp_etiology', 'diagnosis_vfp_iatrogenic', 'diagnosis_vfp_tumor', 'diagnosis_vfp_dse', 'diagnosis_degree_os', 'diagnosis_degree_os_2', 'diagnosis_degree_r', 'diagnosis_degree_r_2', 'diagnosis_degree_b', 'diagnosis_degree_b_2', 'diagnosis_degree_s', 'diagnosis_degree_s_2', 'diagnosis_degree_p', 'diagnosis_degree_p_2', 'diagnosis_degree_l', 'diagnosis_degree_l_2', 'diagnosis_degree_comments']
/Users/isaacbevers/sensein/reproschema-wrapper/b2ai-redcap2rs/activities/d_voice_unilateral_vocal_fold_paralysis/d_voice_unilateral_vocal_fold_paralysis_schema
['diagnosis_alz

In [87]:
print(len(non_matching))
print(non_matching)
print(len(matching_non_item_files))
print(matching_non_item_files)
print(len(multiple_item_files))


2
['q_generic_demographics_complete', 'subjectparticipant_eligible_studies_complete']
0
{}
540


In [88]:
phenotype_dir = "/Users/isaacbevers/sensein/b2ai-wrapper/b2aiprep/src/b2aiprep/prepare/resources/b2ai-data-bids-like-template/phenotype"

def count_items_with_only_descriptions():
    single_entry_fields = {}
    for phenotype_file_name in os.listdir(phenotype_dir):
        if phenotype_file_name.endswith(".json") and phenotype_file_name != "<measurement_tool_name>.json":
            single_entry_fields[phenotype_file_name] = []
            file_path = os.path.join(phenotype_dir, phenotype_file_name)

            # Open and load the JSON file
            with open(file_path, 'r', encoding='utf-8') as file:
                phenotype_file_dict = json.load(file)

            for key in phenotype_file_dict:
                if len(phenotype_file_dict[key]) < 2:
                    single_entry_fields[phenotype_file_name].append(key)
            
    single_entry_fields_count = 0
    for key in single_entry_fields:
        single_entry_fields_count += len(single_entry_fields[key])
        single_entry_fields[key] = len(single_entry_fields[key])
    print(single_entry_fields)
    print(single_entry_fields_count)

count_items_with_only_descriptions()
        

{'vhi10.json': 0, 'vocalFoldParalysis.json': 0, 'alzheimers.json': 28, 'confounders.json': 135, 'demographics.json': 32, 'panas.json': 0, 'bipolar.json': 0, 'dsm5.json': 0, 'airwaystenosis.json': 19, 'enrollment.json': 27, 'parkinsons.json': 37, 'stroop.json': 0, 'laryngealCancer.json': 29, 'adhd.json': 0, 'winograd.json': 0, 'voicePerception.json': 0, 'gad7.json': 0, 'random.json': 0, 'voiceSeverity.json': 0, 'precancerousLesions.json': 23, 'customAffectScale.json': 0, 'depression.json': 0, 'laryngealDystonia.json': 28, 'als.json': 30, 'dyspnea.json': 0, 'vocab.json': 0, 'eligibility.json': 6, 'phq9.json': 0, 'leicester.json': 0, 'benignLesion.json': 5, 'participant.json': 233, 'ptsd.json': 0}
632


In [89]:
# fix options with ___
#